In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
# Create lagged dataset
def create_lagged_dataset(dataset, lag_dict, target_var):
    lagged_dataset = dataset.copy()
    columns_list = list(lagged_dataset.columns)
    data_join = {}
    for column in columns_list:
        if column == 'date':
            continue
        col_lag = lag_dict[column]
        if column == target_var:
            data_join[column] = lagged_dataset[column]
        for n in range(1,col_lag+1):
            data_join[F'{column}_L{n}'] = lagged_dataset[column].shift(n)
    lagged_dataset = pd.concat(data_join.values(), axis=1, ignore_index = True)
    lagged_dataset.columns = data_join.keys()
    return lagged_dataset.dropna() # dropping of NaN values so that regression can be easily done

# Generate Stepped Dataset for Training
## Steps is the number of weeks ahead that we are forecasting, e.g. step=2 is 2 weeks ahead.
## Note step=1 results in no change to dataset, i.e. use generated lagged variables to forecast current. 
def create_stepped_dataset(dataset, step, target_var):
    shifted_dataset = dataset.copy()
    y = shifted_dataset[[target_var]].shift(-step+1)
    if step != 1:
        X = shifted_dataset.iloc[:-step+1, :] # remove the last few rows
    else:
        X = shifted_dataset
    return X.drop(target_var, axis = 1), y.dropna()
## So now target variable (y variable for exploration) is shifted back by 2 weeks. i.e., taking the y-value from 2 weeks later
## and setting it to the current index. So linear regression of y+2 with the current X values. X will have
## a smaller dataset with the last 2 time points removed because of the shift. 

def generate_next_date(list_of_dates, date):
    return list_of_dates[list_of_dates > date].min()

def process_factor_model(X_factor, y_factor, train_end):
    X_train = X_factor.iloc[train_end-120:train_end]
    X_test = X_factor.iloc[train_end:train_end+1]

    y_train = y_factor.loc[X_train.index]
    y_test = y_factor.loc[X_test.index]
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test

In [2]:
us_optimal_lags = dict({
    'infl_rate_diff':14,
    'cons_conf':1,
    'gdp_pct_change':14,
    'int_3m':3,
    'ppi_diff':14,
    'unm_rate':1
})

uk_optimal_lags = dict({
    'infl_rate_diff':11,
    'cons_conf':1,
    'gdp_pct_change':11,
    'int_3m':1,
    'ppi_diff':12,
    'unm_rate':4
})

In [4]:
lstm_hyperparams_dict_healthcare_us = dict(
    {1: dict({
        'units':[96,32,32,32],
        'optimizer':'Nadam',
        'drop_out':[0.1,0.1,0.1,0.1],
        'activation':['linear','linear','tanh','linear'],
        'lr':0.00209452781254191
    }),
     2: dict({
        'units':[96,32,32],
        'optimizer':'Nadam',
        'drop_out':[0.2,0.1,0.2],
        'activation':['linear','linear','tanh'],
        'lr':0.004107775862916715
    }),
     3: dict({
        'units':[96,32,32],
        'optimizer':'Adam',
        'drop_out':[0.2,0.2,0.2],
        'activation':['tanh','tanh','linear'],
        'lr':0.0035406840889284476
    }),
     4: dict({
        'units':[32,32,32],
        'optimizer':'Adam',
        'drop_out':[0.2,0.1,0.2],
        'activation':['tanh','linear','linear'],
        'lr':0.0005842086404315508
    }),
     5: dict({
        'units':[32,32,32],
        'optimizer':'Adam',
        'drop_out':[0.2,0.2,0.1],
        'activation':['sigmoid','tanh','linear'],
        'lr':0.01539768812695964
    }),
     6: dict({
        'units':[64,32,32],
        'optimizer':'Adam',
        'drop_out':[0.2,0.2,0.1],
        'activation':['tanh','linear','linear'],
        'lr':0.001172720778910653
    })}
)

lstm_hyperparams_dict_healthcare_uk = dict(
    {1: dict({
        'units':[32,32],
        'optimizer':'Adam',
        'drop_out':[0.1,0.1],
        'activation':['linear','tanh'],
        'lr':0.04692773350706299
    }),
     2: dict({
        'units':[32,32,32,32],
        'optimizer':'Nadam',
        'drop_out':[0.2,0.2,0.1,0.1],
        'activation':['linear','linear','tanh','tanh'],
        'lr':0.0027559297251538695
    }),
     3: dict({
        'units':[128,32,32,32],
        'optimizer':'Nadam',
        'drop_out':[0.1,0.2,0.2,0.1],
        'activation':['tanh','linear','tanh','tanh'],
        'lr':0.0033904359243909243
    }),
     4: dict({
        'units':[96,32,32,32],
        'optimizer':'Nadam',
        'drop_out':[0.2,0.2,0.2,0.1],
        'activation':['tanh','linear','tanh','tanh'],
        'lr':0.0058921140024817105
    }),
     5: dict({
        'units':[64,32,32],
        'optimizer':'Adagrad',
        'drop_out':[0.2,0.1,0.1],
        'activation':['tanh','sigmoid','linear'],
        'lr':0.001379526955364811
    }),
     6: dict({
        'units':[96,32,32,32],
        'optimizer':'Nadam',
        'drop_out':[0.1,0.2,0.2,0.2],
        'activation':['tanh','linear','tanh','tanh'],
        'lr':0.00023168509056616956
    }),
    7: dict({
        'units':[64,32,32,32],
        'optimizer':'Nadam',
        'drop_out':[0.2,0.2,0.1,0.1],
        'activation':['tanh','tanh','linear','linear'],
        'lr':0.0010137983528640687
    }),
    8: dict({
        'units':[32,32,32],
        'optimizer':'Nadam',
        'drop_out':[0.2,0.1,0.2],
        'activation':['tanh','tanh','tanh'],
        'lr':0.04818160436963496
    }),
    9: dict({
        'units':[128,32],
        'optimizer':'Adam',
        'drop_out':[0.1,0.1],
        'activation':['tanh','linear'],
        'lr':0.004347669530569911
    }),
    10: dict({
        'units':[128,32],
        'optimizer':'Adam',
        'drop_out':[0.2,0.2],
        'activation':['tanh','tanh'],
        'lr':0.0003758643392217273
    }),
    11: dict({
        'units':[128,32,32,32],
        'optimizer':'Adam',
        'drop_out':[0.1,0.2,0.2,0.1],
        'activation':['tanh','tanh','linear','linear'],
        'lr':0.0005964355656236208
    }),
    12: dict({
        'units':[96,32],
        'optimizer':'Nadam',
        'drop_out':[0.1,0.2],
        'activation':['tanh','linear'],
        'lr':0.0038874392172875733
    }),
    13: dict({
        'units':[64,32,32],
        'optimizer':'Adam',
        'drop_out':[0.1,0.1,0.1],
        'activation':['linear','linear','tanh'],
        'lr':0.0003414596058972786
    }),
    14: dict({
        'units':[128,32,32],
        'optimizer':'Adam',
        'drop_out':[0.2,0.1,0.2],
        'activation':['tanh','tanh','linear'],
        'lr':0.0009549466557181202
    })}
)

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV, SGDRegressor
import numpy as np
import math
import keras_tuner as kt
import tensorflow as tf
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam, Adagrad, Nadam
from sklearn.decomposition import PCA
import gc

def overall_function(dataset_link, outcome_var, lag_dict, horizon_num, window_length, lstm_hyperparam_dict):
    dataset = pd.read_csv(dataset_link)
    dataset['ppi_diff'] = dataset['ppi'].diff()
    dataset['infl_rate_diff'] = dataset['infl_rate'].diff()
    dataset = dataset.dropna(subset=['infl_rate_diff'])
    dataset = dataset.drop(columns=['infl_rate','ppi'])
    dataset['date'] = pd.to_datetime(dataset['date']) # converting to date format
    dataset = dataset.sort_values(by='date')
    dataset = dataset.drop(columns = 'date')
    # Creating Lagged and Stepped Datasets
    X_dataset, y_dataset = create_stepped_dataset(create_lagged_dataset(dataset, lag_dict = lag_dict, target_var = outcome_var), step=horizon_num, target_var = outcome_var)
    
    # Initialising the training end
    train_end = window_length
    # train_end = 452
    
    results = []
    
    ## Implement cross-validation split
    tscv = TimeSeriesSplit(n_splits = 5)
    batch_num = 1
    count = 0
    while train_end < len(X_dataset):
        if count == 24:
            batch_num += 1
            count = 0 # Restart count
        # Process data for modeling
        X_train, X_test, y_train, y_test = process_factor_model(X_dataset, y_dataset, train_end)
        
        # For Adding Results
        df_in_loop = y_test.copy()
        
        ### Model 1: Ridge Regression ###
        ridge_cv = RidgeCV(cv = tscv)
        ridge_cv.fit(X_train, y_train)
    
        ridge_model = Ridge(alpha = ridge_cv.alpha_)
        ridge_model.fit(X_train, y_train)
        
        ridgepred = ridge_model.predict(X_test)
        # Adding Linear Model
        df_in_loop[f'ridge'] = ridgepred
        print('Ridge Done')
        
        ### Model 2: PCA ###
        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)

        X_train_pure = X_train
        X_test_pure = X_test
        
        pca = PCA()
        pca.fit(X_train_pure)
        cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
          #to explain more than 85% of the variance
        num_components = np.where(cumulative_variance_ratio >= 0.85)[0][0] + 1 
        pca_new = PCA(n_components=num_components)
        X_train_pca = pca_new.fit_transform(X_train_pure)
        X_train_pca = pd.DataFrame(X_train_pca)
        X_train_pca.columns = X_train_pca.columns.astype(str)

        X_test_pca = pca_new.transform(X_test_pure)
        X_test_pca = pd.DataFrame(X_test_pca)
        X_test_pca.columns = X_test_pca.columns.astype(str)
        
        pure_factor_model = LinearRegression()
        pure_factor_model.fit(X_train_pca, y_train)
        y_pred = pure_factor_model.predict(X_test_pca)
        df_in_loop[f'pca'] = y_pred
        print('PCA Done')
        
        # **Free Memory Before LSTM**
        tf.keras.backend.clear_session()
        gc.collect()
        
        ## Model 3: LSTM ###
        X_train_lstm = np.array(X_train).astype(np.float32)
        X_test_lstm = np.array(X_test).astype(np.float32)
        y_train_lstm = np.array(y_train).astype(np.float32)
        
        num_layers = len(lstm_hyperparam_dict[batch_num]['units'])
        print(f'Using LSTM Hyperparameters from batch {batch_num} which has {num_layers} layers!')
        
        # Building LSTM Model
        lstm_model = Sequential()
        for layer_num in range(num_layers):
            is_last_layer =(layer_num == (num_layers - 1))  # Check if it's the last layer
            lstm_model.add(LSTM(
                units=lstm_hyperparam_dict[batch_num]['units'][layer_num], 
                return_sequences=not is_last_layer,  # Only last layer has return_sequences=False
                input_shape=(X_train_lstm.shape[1], 1) if layer_num == 0 else None,  # Define input shape only for the first layer
                activation=lstm_hyperparam_dict[batch_num]['activation'][layer_num]
            ))
            lstm_model.add(Dropout(lstm_hyperparam_dict[batch_num]['drop_out'][layer_num]))
        # Output Layer
        lstm_model.add(Dense(units=1))
        
        # Compilation
        if lstm_hyperparam_dict[batch_num]['optimizer'] == 'Adam':
            lstm_model.compile(
                optimizer=Adam(learning_rate=lstm_hyperparam_dict[batch_num]['lr']),
                loss='mean_squared_error',
                metrics=[
                    tf.keras.metrics.RootMeanSquaredError()
                ]
            )
        elif lstm_hyperparam_dict[batch_num]['optimizer'] == 'Nadam':
            lstm_model.compile(
                optimizer=Nadam(learning_rate=lstm_hyperparam_dict[batch_num]['lr']),
                loss='mean_squared_error',
                metrics=[
                    tf.keras.metrics.RootMeanSquaredError()
                ]
            )
        else:
            lstm_model.compile(
                optimizer=Adagrad(learning_rate=lstm_hyperparam_dict[batch_num]['lr']),
                loss='mean_squared_error',
                metrics=[
                    tf.keras.metrics.RootMeanSquaredError()
                ]
            )

        # Early stopping callback
        callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

        # Train the model
        lstm_model.fit(
            X_train_lstm, y_train_lstm,
            epochs=10, batch_size=16,
            callbacks=[callback], verbose=0
        )

        lstmpred = lstm_model.predict(X_test_lstm)
        df_in_loop['lstm'] = lstmpred.reshape(-1)  # Ensure correct shape
        print('LSTM Done')
        
        # Add results into loop
        results.append(df_in_loop)
        train_end += 1
        count += 1
        num_remaining_dates = len(X_dataset) - train_end
        print(f'{num_remaining_dates} dates remaining')
        
    # return(results)
    combined_df = pd.concat(results, ignore_index=True)
    return combined_df

In [ ]:
us_results = overall_function("../data/US_consolidated_df.csv", 
                     outcome_var = "infl_rate_diff", 
                     lag_dict= us_optimal_lags, 
                     horizon_num = 1, window_length=120, 
                     lstm_hyperparam_dict=lstm_hyperparams_dict_healthcare_us)
us_results.to_csv("us_results.csv")

Ridge Done
PCA Done
Using LSTM Hyperparameters from batch 1 which has 4 layers!


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 19s 90ms/step - loss: 2.7486e-04 - root_mean_squared_error: 0.0157
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - loss: 1.6247e-05 - root_mean_squared_error: 0.0040
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - loss: 2.1915e-05 - root_mean_squared_error: 0.0047
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - loss: 2.4565e-05 - root_mean_squared_error: 0.0049
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - loss: 1.3975e-05 - root_mean_squared_error: 0.0037
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - loss: 1.6165e-05 - root_mean_squared_error: 0.0040
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - loss: 1.5594e-05 - root_mean_squared_error: 0.0039
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - loss: 1.1893e-05 - root_mean_squared_error: 0.0034
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - loss: 1.6403e-05 - root_mean_squared_error: 0.0040
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - loss: 1.6139e-05 - root_mean_

C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 19s 81ms/step - loss: 2.4201e-04 - root_mean_squared_error: 0.0148
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - loss: 2.0906e-05 - root_mean_squared_error: 0.0046
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - loss: 1.7535e-05 - root_mean_squared_error: 0.0042
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - loss: 1.8769e-05 - root_mean_squared_error: 0.0043
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - loss: 1.8100e-05 - root_mean_squared_error: 0.0042
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - loss: 1.6122e-05 - root_mean_squared_error: 0.0040
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - loss: 1.9863e-05 - root_mean_squared_error: 0.0044
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - loss: 1.4248e-05 - root_mean_squared_error: 0.0038
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - loss: 1.1715e-05 - root_mean_squared_error: 0.0034
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - loss: 1.5621e-05 - root_mean_

...

Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 12s 67ms/step - loss: 2.3051e-04 - root_mean_squared_error: 0.0151
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 6.1118e-05 - root_mean_squared_error: 0.0078
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 2.5840e-05 - root_mean_squared_error: 0.0051
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - loss: 3.5444e-05 - root_mean_squared_error: 0.0059
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 1.4121e-05 - root_mean_squared_error: 0.0038
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - loss: 1.4178e-05 - root_mean_squared_error: 0.0037
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 1.2129e-05 - root_mean_squared_error: 0.0035
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 1.3438e-05 - root_mean_squared_error: 0.0037
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 1.5045e-05 - root_mean_squared_error: 0.0039
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - loss: 9.6236e-06 - root_mean_

C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 13s 66ms/step - loss: 4.7067e-04 - root_mean_squared_error: 0.0213
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 6.2697e-05 - root_mean_squared_error: 0.0079
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 4.8330e-05 - root_mean_squared_error: 0.0069
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 1.8266e-05 - root_mean_squared_error: 0.0043
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - loss: 2.4783e-05 - root_mean_squared_error: 0.0050
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - loss: 1.6962e-05 - root_mean_squared_error: 0.0041
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - loss: 1.6507e-05 - root_mean_squared_error: 0.0040
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 1.6579e-05 - root_mean_squared_error: 0.0041
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 6.4627e-06 - root_mean_squared_error: 0.0025
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 1.1520e-05 - root_mean_

C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 13s 65ms/step - loss: 1.3718e-04 - root_mean_squared_error: 0.0116
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 4.5889e-05 - root_mean_squared_error: 0.0068
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 2.8841e-05 - root_mean_squared_error: 0.0054
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 1.4451e-05 - root_mean_squared_error: 0.0038
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - loss: 1.2843e-05 - root_mean_squared_error: 0.0036
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 1.0715e-05 - root_mean_squared_error: 0.0033
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 1.1910e-05 - root_mean_squared_error: 0.0034
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 8.6706e-06 - root_mean_squared_error: 0.0029
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 1.0728e-05 - root_mean_squared_error: 0.0032
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - loss: 1.0042e-05 - root_mean_

In [ ]:
uk_results = overall_function("../data/UK_consolidated_df.csv", 
                     outcome_var = "infl_rate_diff", 
                     lag_dict= uk_optimal_lags, 
                     horizon_num = 1, window_length=120,
                     lstm_hyperparam_dict=lstm_hyperparams_dict_healthcare_uk)
uk_results.to_csv("uk_results.csv")

Ridge Done
PCA Done
Using LSTM Hyperparameters from batch 1 which has 2 layers!


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 813ms/step
LSTM Done
333 dates remaining
Ridge Done
PCA Done
Using LSTM Hyperparameters from batch 1 which has 2 layers!


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 770ms/step
LSTM Done
332 dates remaining
Ridge Done
PCA Done
Using LSTM Hyperparameters from batch 1 which has 2 layers!


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
LSTM Done
88 dates remaining
Ridge Done
PCA Done
Using LSTM Hyperparameters from batch 11 which has 4 layers!


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 956ms/step
LSTM Done
87 dates remaining
Ridge Done
PCA Done
Using LSTM Hyperparameters from batch 11 which has 4 layers!


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 991ms/step
LSTM Done
86 dates remaining


Hyperparameter Tuning for LSTM

In [13]:
from tensorflow.keras.optimizers import Adam, Adagrad, Nadam
def build_lstm_model(hp):
    lstm_model = Sequential()
    # Select the number of LSTM layers (2, 3, or 4)
    num_layers = hp.Choice('num_layers', [2, 3, 4])
    # Choose optimizer
    optimizer_name = hp.Choice('optimizer', ['Adam', 'Adagrad', 'Nadam'])
    optimizers = {
        "Adam": Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='LOG')),
        "Adagrad": Adagrad(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='LOG')),
        "Nadam": Nadam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='LOG'))
    }
    # Layer 1
    first_layer_units = hp.Int('lstm_units_1', min_value=32, max_value=128, step=32)
    lstm_model.add(LSTM(
        units=first_layer_units,
        return_sequences=True,
        input_shape=(X_train_lstm.shape[1], 1),
        activation=hp.Choice(f'activation_1', ['tanh', 'sigmoid', 'linear'])
    ))
    lstm_model.add(Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.3, step=0.1)))
    
    # Add remaining layers with non-increasing units
    previous_units = first_layer_units  # Track previous layer units
    
    for i in range(1, num_layers):
        available_units = [u for u in [32, 64, 96, 128] if u <= previous_units]  # Only allow non-increasing sizes
        current_units = hp.Choice(f'lstm_units_{i+1}', available_units)

        lstm_model.add(LSTM(
            units=current_units,
            return_sequences=True if i < num_layers - 1 else False,  
            activation=hp.Choice(f'activation_{i+1}', ['tanh', 'sigmoid', 'linear'])
        ))
        lstm_model.add(Dropout(hp.Float(f'dropout_{i+1}', min_value=0.1, max_value=0.3, step=0.1)))
        previous_units = current_units  # Update tracking variable
    
    # Output Layer
    lstm_model.add(Dense(units=1))
    
    # Compile the model
    lstm_model.compile(
        optimizer=optimizers[optimizer_name],
        loss='mean_squared_error',
        metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )
    return lstm_model

def tune_lstm(X_train_lstm, y_train_lstm, batch_num, country, save_path="best_lstm_model"):
    tuner = kt.BayesianOptimization(
        build_lstm_model,
        objective='val_loss',
        max_trials=20,
        executions_per_trial=1,
        directory=f'lstm_tuning_batch_{batch_num}_{country}',
        project_name='fund_forecasting'
    )

    # Perform the search with batch_size as a hyperparameter to tune
    tuner.search(
        X_train_lstm, y_train_lstm,
        epochs=10,
        validation_split=0.2,
        batch_size=16,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]
    )

    # Get the best hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    
    # Print the best hyperparameters
    print("Best Hyperparameters:", best_hps.values)
    return best_hps

In [17]:
uk_df = pd.read_csv("../data/UK_consolidated_df.csv")
uk_df['ppi_diff'] = uk_df['ppi'].diff()
uk_df['infl_rate_diff'] = uk_df['infl_rate'].diff()
uk_df = uk_df.dropna(subset=['infl_rate_diff'])
uk_df = uk_df.drop(columns=['infl_rate','ppi'])

lagged_df = create_lagged_dataset(uk_df,lag_dict=us_optimal_lags,target_var="infl_rate_diff")
covariates, y = create_stepped_dataset(lagged_df, step = 1, target_var = 'infl_rate_diff')

train_end = 120
# df_end = list_of_dates.max()

hyperparam_lst = []
batch_num = 1

# First batch
test_date = train_end + 1
X_train, X_test, y_train, y_test = process_factor_model(covariates, y, train_end)
X_train_lstm = np.array(X_train).astype(np.float32)
X_test_lstm = np.array(X_test).astype(np.float32)
y_train_lstm = np.array(y_train).astype(np.float32)
best_model_params = tune_lstm(X_train_lstm, y_train_lstm, batch_num = batch_num, country = "UK")
hyperparam_lst.append(best_model_params)

counter = 1
while test_date <= len(covariates):
    test_date = train_end + 1
    if counter != 24: # Still same year: skip and repeat until the year is the next year
        train_end = test_date
        print(f"Counter is {counter}. Repeating until following year is obtained")
        counter += 1
        continue
    else:
        print(f"Counter is {counter}. Conducting Tuning")
        # Process data for modeling
        X_train, X_test, y_train, y_test = process_factor_model(covariates, y, train_end)
        X_train_lstm = np.array(X_train).astype(np.float32)
        X_test_lstm = np.array(X_test).astype(np.float32)
        y_train_lstm = np.array(y_train).astype(np.float32)
        best_model_params = tune_lstm(X_train_lstm, y_train_lstm, batch_num= batch_num+1, country = "UK")
        hyperparam_lst.append(best_model_params)
        train_end = test_date # after tuning, expand the training set
        counter = 1 # Reinitialise.... Every year 1 new hyperparameter is introduced
        batch_num += 1

Reloading Tuner from lstm_tuning_batch_1_UK\fund_forecasting\tuner0.json
Best Hyperparameters: {'num_layers': 2, 'optimizer': 'Adam', 'learning_rate': 0.04692773350706299, 'lstm_units_1': 32, 'activation_1': 'linear', 'dropout_1': 0.1, 'lstm_units_2': 32, 'activation_2': 'tanh', 'dropout_2': 0.1, 'lstm_units_3': 32, 'activation_3': 'tanh', 'dropout_3': 0.1, 'lstm_units_4': 32, 'activation_4': 'linear', 'dropout_4': 0.1}
Counter is 1. Repeating until following year is obtained
Counter is 2. Repeating until following year is obtained
Counter is 3. Repeating until following year is obtained
Counter is 4. Repeating until following year is obtained
Counter is 5. Repeating until following year is obtained
Counter is 6. Repeating until following year is obtained
Counter is 7. Repeating until following year is obtained
Counter is 8. Repeating until following year is obtained
Counter is 9. Repeating until following year is obtained
Counter is 10. Repeating until following year is obtained
Count

# Processing of Results

In [1]:
import pandas as pd
uk_results = pd.read_csv('uk_ml_results.csv').drop(columns='Unnamed: 0')
us_results = pd.read_csv('us_ml_results.csv').drop(columns='Unnamed: 0')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def process_and_plot_results(pdf_link: str, results: pd.DataFrame):
    df = pd.read_csv(pdf_link)

    # Compute inflation rate difference and process data
    df['infl_rate_diff'] = df['infl_rate'].diff()
    df['ppi_diff'] = df['ppi'].diff()
    df = df.dropna(subset=['infl_rate_diff'])
    df = df.drop(columns=['ppi'])
    
    df_new = df.iloc[(len(df)-len(results)-1):-1][['infl_rate']]
    true_dates = df.iloc[(len(df)-len(results)):][['date']]
    df_new_combined = pd.concat([df_new.reset_index(drop=True), 
                                 results.reset_index(drop=True)], axis=1)

    # Compute adjusted values
    df_new_combined['actual'] = df_new_combined['infl_rate'] + df_new_combined['infl_rate_diff']
    df_new_combined['ridge'] = df_new_combined['infl_rate'] + df_new_combined['ridge']
    df_new_combined['pca'] = df_new_combined['infl_rate'] + df_new_combined['pca']
    df_new_combined['lstm'] = df_new_combined['infl_rate'] + df_new_combined['lstm']
    df_new_combined_final = df_new_combined[['actual', 'ridge', 'pca', 'lstm']]
    combined_results = pd.concat([true_dates.reset_index(drop=True), 
                                 df_new_combined_final.reset_index(drop=True)], axis=1)

    # Convert date column to datetime format
    combined_results["date"] = pd.to_datetime(combined_results["date"])

    return combined_results

# Load results for US and UK
us_results_overall = process_and_plot_results("../data/US_consolidated_df.csv", us_results)
uk_results_overall = process_and_plot_results("../data/UK_consolidated_df.csv", uk_results)

,date,actual,ridge,pca,lstm
0,2014-08-01,-0.000160,0.000137,1.269404e-03,0.005530
1,2014-09-01,0.000072,0.000941,1.524584e-03,0.002264
2,2014-10-01,-0.000198,-0.000858,1.977709e-03,-0.000158
3,2014-11-01,-0.001883,-0.000148,2.053533e-03,-0.001364
4,2014-12-01,-0.003085,-0.002825,-9.373446e-07,0.001878
...,...,...,...,...,...
122,2024-10-01,0.002265,0.000687,2.044770e-03,0.002072
123,2024-11-01,0.002805,0.003757,2.737529e-03,0.003313
124,2024-12-01,0.003647,0.001845,2.651288e-03,0.003634
125,2025-01-01,0.004669,0.002093,2.621717e-03,0.002896


In [9]:
import numpy as np

def calculate_rmse(actual, predicted):
    return np.sqrt(np.mean((actual - predicted) ** 2))

# RMSE for Ridge
rmse_ridge_us = calculate_rmse(us_results_overall['actual'], us_results_overall['ridge'])

# RMSE for PCA
rmse_pca_us = calculate_rmse(us_results_overall['actual'], us_results_overall['pca'])

# RMSE for LSTM
rmse_lstm_us = calculate_rmse(us_results_overall['actual'], us_results_overall['lstm'])

# Print RMSE values
print(f"RMSE for US Ridge: {rmse_ridge_us}")
print(f"RMSE for US PCA: {rmse_pca_us}")
print(f"RMSE for US LSTM: {rmse_lstm_us}")

# RMSE for Ridge
rmse_ridge_uk = calculate_rmse(uk_results_overall['actual'], uk_results_overall['ridge'])

# RMSE for PCA
rmse_pca_uk = calculate_rmse(uk_results_overall['actual'], uk_results_overall['pca'])

# RMSE for LSTM
rmse_lstm_uk = calculate_rmse(uk_results_overall['actual'], uk_results_overall['lstm'])

# Print RMSE values
print(f"RMSE for UK Ridge: {rmse_ridge_uk}")
print(f"RMSE for UK PCA: {rmse_pca_uk}")
print(f"RMSE for UK LSTM: {rmse_lstm_uk}")

RMSE for US Ridge: 0.0033860336516231956
RMSE for US PCA: 0.0028605306519446485
RMSE for US LSTM: 0.0037676910886561936
RMSE for UK Ridge: 0.0035121137690686086
RMSE for UK PCA: 0.0042115865579569105
RMSE for UK LSTM: 0.005180380671975229


In [ ]:
def plot_us_uk_comparison(us_results, uk_results):
    # Create a 3x2 subplot grid (3 rows, 2 columns)
    fig, axs = plt.subplots(3, 2, figsize=(16, 18), sharey='all')

    # Plot US results in the first column
    axs[0, 0].plot(us_results["date"], us_results["actual"], label="Actual", color='blue', linewidth=2)
    axs[0, 0].plot(us_results["date"], us_results["ridge"], label="Ridge", color='red')
    axs[0, 0].set_ylabel("Value", fontsize=12)
    axs[0, 0].set_title("US - Actual vs Ridge", fontsize=14, fontweight = 'bold')
    axs[0, 0].grid(True)  # Add grid
    axs[0, 0].set_xticks([])  # Remove x-ticks

    axs[1, 0].plot(us_results["date"], us_results["actual"], label="Actual", color='blue', linewidth=2)
    axs[1, 0].plot(us_results["date"], us_results["pca"], label="PCA", color='green')
    axs[1, 0].set_ylabel("Value", fontsize=12)
    axs[1, 0].set_title("US - Actual vs PCA", fontsize=14, fontweight = 'bold')
    axs[1, 0].grid(True)  # Add grid
    axs[1, 0].set_xticks([])  # Remove x-ticks

    axs[2, 0].plot(us_results["date"], us_results["actual"], label="Actual", color='blue', linewidth=2)
    axs[2, 0].plot(us_results["date"], us_results["lstm"], label="LSTM", color='orange')
    axs[2, 0].set_xlabel("Date", fontsize=12)
    axs[2, 0].set_ylabel("Value", fontsize=12)
    axs[2, 0].set_title("US - Actual vs LSTM", fontsize=14, fontweight = 'bold')
    axs[2, 0].grid(True, axis='y')  # Add grid

    # Plot UK results in the second column
    axs[0, 1].plot(uk_results["date"], uk_results["actual"], label="Actual", color='blue', linewidth=2)
    axs[0, 1].plot(uk_results["date"], uk_results["ridge"], label="Ridge", color='red')
    axs[0, 1].set_ylabel("Value", fontsize=12)
    axs[0, 1].set_title("UK - Actual vs Ridge", fontsize=14, fontweight = 'bold')
    axs[0, 1].grid(True)  # Add grid
    axs[0, 1].set_xticks([])  # Remove x-ticks

    axs[1, 1].plot(uk_results["date"], uk_results["actual"], label="Actual", color='blue', linewidth=2)
    axs[1, 1].plot(uk_results["date"], uk_results["pca"], label="PCA", color='green')
    axs[1, 1].set_ylabel("Value", fontsize=12)
    axs[1, 1].set_title("UK - Actual vs PCA", fontsize=14, fontweight = 'bold')
    axs[1, 1].grid(True)  # Add grid
    axs[1, 1].set_xticks([])  # Remove x-ticks
    
    axs[2, 1].plot(uk_results["date"], uk_results["actual"], label="Actual", color='blue', linewidth=2)
    axs[2, 1].plot(uk_results["date"], uk_results["lstm"], label="LSTM", color='orange')
    axs[2, 1].set_xlabel("Date", fontsize=12)
    axs[2, 1].set_ylabel("Value", fontsize=12)
    axs[2, 1].set_title("UK - Actual vs LSTM", fontsize=14, fontweight = 'bold')
    axs[2, 1].grid(True, axis='y')  # Add grid

    # Define a single custom legend for all subplots
    handles = [
        plt.Line2D([0], [0], color='blue', lw=2, label='Actual'),
        plt.Line2D([0], [0], color='red', lw=2, label='Ridge'),
        plt.Line2D([0], [0], color='green', lw=2, label='PCA'),
        plt.Line2D([0], [0], color='orange', lw=2, label='LSTM')
    ]
    labels = ['Actual', 'Ridge', 'PCA', 'LSTM']

    # Add a common legend with the custom handles
    fig.legend(handles, labels, loc='center right', ncol=1, bbox_to_anchor=(1.1, 0.5), fontsize=16)

    # Rotate x-axis labels for all subplots
    for ax in axs.flat:
        ax.set_xticks(ax.get_xticks())
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

    # Adjust layout to prevent overlap
    plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust title and layout

    # Show plot
    plt.show()
    
plot_us_uk_comparison(us_results_overall, uk_results_overall)